<a href="https://colab.research.google.com/github/icepenguinz/penguinclassifier/blob/main/penguin_classifier4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Mount google drive and install libraries.


In [7]:
from google.colab import drive

# mount google drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/colab_projects/penguin_classifier4
%cd /content/drive/MyDrive/colab_projects/penguin_classifier4


Mounted at /content/drive
/content/drive/MyDrive/colab_projects/penguin_classifier4


In [1]:
from IPython.display import HTML, display

# line wrap in colab for readability

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
# Install libraries
! pip install torch torchvision streamlit pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19580 sha256=e42d9c10a2f3c01f1954e5b167b00c1d2ddb66ec9d80147da490b9cd5b597186
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzloca

In [4]:
import torch
torch.cuda.is_available()

True

# Dataset

Image data was downloaded from google images with the help of a chrome extension (Download all images). The species of penguins include gentoo, adelie, chinstrap, miniso penguins and piplup (pokemon).

# Code

In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from PIL import Image

In [40]:
# Define CNN model
class PenguinCNN(nn.Module):
    def __init__(self, num_classes): #edit
        super(PenguinCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), #3 inchannels, 32 outchannels
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(128),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Data normalization for testing
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Load the dataset
data_dir = '/content/drive/MyDrive/colab_projects/penguin_classifier4/data'
class_names = os.listdir(data_dir)
# print(class_names)
class_to_index = {class_name: index for index, class_name in enumerate(class_names)}
# print(class_to_index)
image_paths = []
labels = []

for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image_paths.append(image_path)
            labels.append(class_to_index[class_name])

# Train test split
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size = 0.3, random_state=42)

class PenguinDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[index]
        # print(label)

        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = PenguinDataset(train_image_paths, train_labels, transform=transform_train)
test_dataset = PenguinDataset(test_image_paths, test_labels, transform=transform_test)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

# Initialize the model and loss function, optimizer
device = torch.device('cuda')
num_classes = len(set(train_labels))
model = PenguinCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Compute the number of correct predictions
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')


# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/colab_projects/penguin_classifier4/penguin_classifier.pth')



Epoch 1/5, Loss: 1.3986, Accuracy: 0.4576
Epoch 2/5, Loss: 1.3732, Accuracy: 0.4925
Epoch 3/5, Loss: 1.1721, Accuracy: 0.5474
Epoch 4/5, Loss: 1.1679, Accuracy: 0.5411
Epoch 5/5, Loss: 1.0931, Accuracy: 0.5436


In [ ]:
# How to increase accuracy

To streamlit

In [30]:
print(class_names)

['chinstrap_penguins', 'piplup', 'adelie_penguins', 'gentoo_penguins', 'miniso_penguins']


In [41]:
%%writefile app.py

import streamlit as st

# Define class names
class_names = ['Chinstrap Penguins', 'Piplup', 'Adelie Penguins', 'Gentoo Penguins', 'Miniso Penguins']
class_to_index= {class_name: index for index, class_name in enumerate(class_names)}


# Define CNN model
class PenguinCNN(nn.Module):
    def __init__(self, num_classes):
        super(PenguinCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(128),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load the trained model
model = PenguinCNN(num_classes=len(class_names))
model.load_state_dict(torch.load('penguin_classifier.pth', map_location=torch.device('cpu')))
model.eval()

# Function to make predictions
def predict_species(image):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class_index = predicted.item()
    predicted_class_name = class_names[predicted_class_index]
    return predicted_class_name

# Streamlit app
def main():
    st.title('Penguin Image Classifier')
    uploaded_file = st.file_uploader('Upload an image of a penguin', type=['jpg', 'jpeg', 'png'])

    if uploaded_file is not None:
        st.image(uploaded_file, caption='Uploaded Image.', use_column_width=True)
        st.write('')
        st.write('Classifying...')

        # Make a prediction
        species = predict_species(uploaded_file)
        st.write(f'Prediction: {species}')

if __name__ == '__main__':
    main()


Overwriting app.py


In [32]:
# !npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/drive/MyDrive/colab_projects/penguin_classifier4/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/drive/MyDrive/colab_projects/penguin_classifier4/package.json'
npm WARN penguin_classifier4 No description
npm WARN penguin_classifier4 No repository field.
npm WARN penguin_classifier4 No README data
npm WARN penguin_classifier4 No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.177s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [36]:
# import urllib
# print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.126.84.70


In [37]:
# !streamlit run app.py &>/content/logs.txt &

In [38]:
# !npx localtunnel --port 8501

npx: installed 22 in 2.368s
your url is: https://heavy-emus-grin.loca.lt
^C
